In [12]:
#recognize words: not "-', don't make difference between some similar words
#maybe use third parties vectorizer :)
#after exlusion of weird words ^^
#requires build a list of only words first


In [13]:
import gensim

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
train_df = pd.read_csv("data/training/data_train.csv", dtype='unicode')

In [15]:
def getListCategories(train_df):
    listCat=[]
    for i in range(1,len (train_df)):
        category = str(train_df['category_lvl_1'][i])+"/"+str(train_df['category_lvl_2'][i])+"/"+str(train_df['category_lvl_3'][i])
        if category not in listCat:
            listCat.append(category)
        
    print("number of categories")
    print(len(listCat))
    return listCat
    

In [68]:
def special_match(strg, search=re.compile(r'[^a-zA-Z0-9.]').search):
    return not bool(search(strg))

def is_code(word):
    if (not word.isalpha()) & (not word.isdigit()) & (special_match(word)) &(not is_measure(word)) :
        return True
    
    return False

def compressSeq(token):
    string=''
    for i in range(len(token)):
        if token[i].isdigit():
            string+='0'
        elif token[i].isalpha():
            string+='a'                        
        elif token[i] in ["?",".", ";", ",", "!",":"]:
            string+='.'
        else:
            string+='?'
        
    string = string.replace("0''", "0aa" )
    i=0
    while (i<len(string)-1):
        if string[i] == string[i+1]:
            string = string.replace(string[i]+string[i+1], string[i])
        else:
            i+=1                             
    return string

def is_measure(word):
    if (not word.isalpha()) & (not word.isdigit()):
        if compressSeq(word) == "0a":
            return True
    return False

def process_word(token):
    token = token.lower()
    if token[len(token)-1] == 's':
        token = token[:len(token)-1] 
    return token


In [17]:
def getListVocabulary(data):    
    #Build a dictionnary with gensim: every word with id associated
    #First remove all codes and measurements and make a list of string with the remaining
        
    texts = [[word.lower() for word in title.split(" ") if process_word(word) == 1 ] for title in train_df['title'] ]
    
    dictionary = gensim.corpora.Dictionary(texts)
    dictionary.save('dict')  
                
    print("size of dictionnary")
    print(len(dictionary))
    return dictionary
        

In [19]:
listVoc=getListVocabulary(train_df)
listCat=getListCategories(train_df)

size of dictionnary
19633
number of categories
204


In [61]:
def embeddVector(train_df, i, listVoc, listCat):
    
    title = train_df['title'][i]
    category = str(train_df['category_lvl_1'][i])+"/"+ str(train_df['category_lvl_2'][i])+"/"+ str(train_df['category_lvl_3'][i])
    words = title.split(" ")
    size = len(listVoc) +  len(listCat)
    vector=np.zeros((size + 2))
    for i in range(len(listVoc)):
        for word in words:
            if process_word(word) == listVoc[i]:
                vector[i] = 1
                
    for i in range(len(listCat)):
        if category == listCat[i]:
            vector[len(listVoc) + 1 + i] = 1
        
    
    nbCodes = 0
    nbMeasures=0
    for word in words:
        if is_code(word):
            nbCodes+=1
            
        
        if is_measure(word):
            nbMeasures+=1
            
            
    vector[size +1] = nbCodes
    vector[size ] = nbMeasures
    return vector    

In [69]:
print(train_df.head(10))
for i in range(50):
    print(embeddVector(train_df, i, listVoc, listCat))


  country              sku_id  \
0      my    AD674FAASTLXANMY   
1      my  AE068HBAA3RPRDANMY   
2      my  AN680ELAA9VN57ANMY   
3      my  AN957HBAAAHDF4ANMY   
4      my    AR511HBAXNWAANMY   
5      my    AS575ELCMZ4WANMY   
6      my  AS727ELAA9LLV1ANMY   
7      my  BU512HBAA4WUVTANMY   
8      my    CL787ELAW29LANMY   
9      my  CO633HLAABREKOANMY   

                                               title  \
0       Adana Gallery Suri Square Hijab – Light Pink   
1  Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz   
2  Andoer 150cm Cellphone Smartphone Mini Dual-He...   
3  ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...   
4  Argital Argiltubo Green Clay For Face and Body...   
5  Asus TP300LJ-DW004H Transformer Book Flip 4GB ...   
6  NG-40C Ring-Shaped 40W 3166lm 5400K Macro Phot...   
7            Buytra Exfoliating Peel Foot Mask 1Pair   
8  CLiPtec OCC121 Slim Flat USB 3.0 Extension Cab...   
9  McDonald's Coke Can Glass Limited Edition 12oz...   

                   

In [10]:
def cleanhtml(raw_html):
    result = re.sub(re.compile('<.*?>'), ' ', raw_html)
    return result
data_df['short_description_clear_html'] = data_df.apply(lambda row: cleanhtml(row['short_description']), axis=1)

NameError: name 'data_df' is not defined

In [26]:
print(len(listVoc))

19633


In [33]:
print(type(listVoc))

<class 'gensim.corpora.dictionary.Dictionary'>
